In [2]:
%pip install langsmith langchain openai faiss-cpu --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langsmith import traceable
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks.tracers.langchain import LangChainTracer
from langchain.chains import LLMChain, RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os

In [ ]:
tracer = LangChainTracer(project_name="ServiceNow QA Evaluation")
load_dotenv() 
os.environ["LANGCHAIN_API_KEY"] = "LANGSMITH_API_KEY"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
#os.environ["LANGCHAIN_PROJECT"] = "youtube-rag"

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_index = FAISS.load_local("/content/drive/MyDrive/my_project/faiss_store", embedding_model, allow_dangerous_deserialization=True)
retriever = faiss_index.as_retriever(search_kwargs={"k": 5})

def run_qa(query: str):
    llm = OpenAI(temperature=0)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )
    return qa_chain(query)

result = run_qa("What is CMDB in ServiceNow?")
print("Answer:", result["result"])
print("Sources:", [doc.metadata["source"] for doc in result["source_documents"]])
